In [4]:
import os
if 'notebooks' in os.getcwd():
    os.chdir('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Visualizing Data

In [5]:
train_df = pd.read_csv('data/train.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
target_df = train_df[['PassengerId','Survived']]

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
train_df['Age'] = imputer.fit_transform(train_df[['Age']])

In [9]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
matrix_sex = encoder.fit_transform(train_df[['Sex']]).toarray()
column_names_sex = ['Female', 'Male']

for i in range(len(matrix_sex.T)):
    train_df[column_names_sex[i]] = matrix_sex.T[i]

In [10]:
# matrix_embarked = encoder.fit_transform(train_df[['Embarked']]).toarray()
# column_names_embarked = ['C','S','Q']

# for i in range(len(matrix_embarked.T)):
#     train_df[column_names_embarked[i]] = matrix_embarked.T[i]
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  Female       891 non-null    float64
 13  Male         891 non-null    float64
dtypes: float64(4), int64(5), object(5)
memory usage: 97.6+ KB


In [11]:
numerical_df = train_df.select_dtypes(include=['int64','float64'])
numerical_df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Female,Male
0,1,0,3,22.0,1,0,7.2500,0.0,1.0
1,2,1,1,38.0,1,0,71.2833,1.0,0.0
2,3,1,3,26.0,0,0,7.9250,1.0,0.0
3,4,1,1,35.0,1,0,53.1000,1.0,0.0
4,5,0,3,35.0,0,0,8.0500,0.0,1.0


In [12]:
target_correlation = abs(numerical_df.corrwith(target_df['Survived']))\
    .sort_values()

In [25]:
target_correlation.tail(7)

Age         0.069809
Parch       0.081629
Fare        0.257307
Pclass      0.338481
Female      0.543351
Male        0.543351
Survived    1.000000
dtype: float64

In [30]:
most_correlated_columns = target_correlation\
    .tail(5)\
    .index[:-1]

In [31]:
X = train_df[most_correlated_columns]
y = target_df['Survived']

## Using Linear Models

In [32]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [33]:
model.fit(X,y)

LogisticRegression()

In [34]:
model.coef_

array([[ 0.00160119, -0.89580151,  1.2906234 , -1.29058912]])

In [35]:
y_pred = model.predict(X)

In [36]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [22]:
test_df = pd.read_csv("data/test.csv")

In [23]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [40]:
test_df_changesex = test_df

encoder = OneHotEncoder()
matrix_sex_test = encoder.fit_transform(test_df_changesex[['Sex']]).toarray()
column_names_sex = ['Female', 'Male']

for i in range(len(matrix_sex_test.T)):
    test_df_changesex[column_names_sex[i]] = matrix_sex_test.T[i]

In [43]:
X_test = test_df_changesex[most_correlated_columns]
X_test

,Fare,Pclass,Female,Male
0,7.8292,3,0.0,1.0
1,7.0000,3,1.0,0.0
2,9.6875,2,0.0,1.0
3,8.6625,3,0.0,1.0
4,12.2875,3,1.0,0.0
...,...,...,...,...
413,8.0500,3,0.0,1.0
414,108.9000,1,1.0,0.0
415,7.2500,3,0.0,1.0
416,8.0500,3,0.0,1.0


In [44]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Fare    417 non-null    float64
 1   Pclass  418 non-null    int64  
 2   Female  418 non-null    float64
 3   Male    418 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 13.2 KB


In [45]:
X_test_replace = X_test.replace(np.nan, 0)

In [46]:
X_test_replace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Fare    418 non-null    float64
 1   Pclass  418 non-null    int64  
 2   Female  418 non-null    float64
 3   Male    418 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 13.2 KB


In [47]:
y_pred_test = model.predict(X_test_replace)

In [49]:
submission = pd.Series(
    data = y_pred_test,
    index = test_df["PassengerId"],
    name = "Survived"
)

In [50]:
submission.to_csv("data/submission.csv")